In [1]:
import cv2

In [3]:
# Load image
img = cv2.imread("C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/IMAGES TO TEST SHARPNESS/IMAGE3.jpg")

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Increase contrast using Histogram Equalization
equalized = cv2.equalizeHist(gray)

# Apply a sharpening kernel
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
sharp = cv2.filter2D(equalized, -1, kernel)

# Save the result
cv2.imwrite("C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/IMAGES TO TEST SHARPNESS/IMAGE3_ENHANCED.jpg", sharp)

True

# ✅Good Model to Upscale

Model name is Upscayl

In [11]:
import cv2

img = cv2.imread("C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/IMAGES TO TEST SHARPNESS/IMAGE3_upscayl_5x_upscayl-lite-4x.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
equalized = cv2.equalizeHist(gray)
cv2.imwrite("C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/IMAGES TO TEST SHARPNESS/IMAGE3_upscayl_upscaled.png", equalized)



True

## Testing

In [12]:
import cv2
import os

# Set your folder with original barcode images
input_folder = "C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/UPSCAYL/Barcodes"        # Change this to your folder name
output_folder = "C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/UPSCAYL/Preprocessed"   # Folder where processed images will be saved

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Supported image types
valid_extensions = [".jpg", ".jpeg", ".png", ".bmp"]

# Loop through all files
for filename in os.listdir(input_folder):
    if any(filename.lower().endswith(ext) for ext in valid_extensions):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)
        
        if img is None:
            print(f"❌ Could not read {filename}")
            continue
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        equalized = cv2.equalizeHist(gray)
        
        # Save processed image
        output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + "_upscaled.jpg")
        cv2.imwrite(output_path, equalized)
        print(f"✅ Processed: {filename} → {output_path}")


✅ Processed: IMAGE1.jpg → C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/UPSCAYL/Preprocessed\IMAGE1_upscaled.jpg
✅ Processed: IMAGE2.jpg → C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/UPSCAYL/Preprocessed\IMAGE2_upscaled.jpg
✅ Processed: IMAGE3.jpg → C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/UPSCAYL/Preprocessed\IMAGE3_upscaled.jpg
✅ Processed: IMAGE4.jpg → C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/UPSCAYL/Preprocessed\IMAGE4_upscaled.jpg
